In [ ]:
import xarray
import numpy as np
import pandas as pd

In [ ]:
# ### States
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)
luh2_states_worldwide = luh2_states.isel(time=slice(1000, 1173))
luh2_states_worldwide['time'] = pd.date_range(start="1850-01-01", end="2022-01-01", freq='YS')

# ### Transition
transition_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/transitions.nc'
luh2_trans = xarray.open_dataset(transition_LUH2, engine="netcdf4", decode_times=False)
luh2_trans_worldwide = luh2_trans.isel(time=slice(1000, 1173))
luh2_trans_worldwide['time'] = pd.date_range(start='1850-01-01', end='2021-01-01', freq='YS')
luh2_trans_worldwide = luh2_trans_worldwide.drop(['primf_to_urban','secdf_to_urban', 'secdn_to_urban', 'primf_bioh', 'primn_bioh','secmf_bioh','secyf_bioh','secnf_bioh', "urban_to_secdf", "urban_to_secdn","urban_to_c3ann","urban_to_c4ann", "urban_to_c3per",
    "urban_to_c4per", "urban_to_c3nfx", "urban_to_pastr", "c3ann_to_urban", "c4ann_to_urban", "c3per_to_urban", "c4per_to_urban", "c3nfx_to_urban", "pastr_to_urban", "range_to_urban"])

In [ ]:
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)

fao_stat = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/fao_all_data.xlsx", engine="openpyxl")
fao_stat.dropna(axis=0, inplace=True)
fao_stat_area_harv = fao_stat[fao_stat['Element']=="Area harvested"]
fao_stat_area_harv.insert(9, 'True Value', fao_stat_area_harv['Value']/100, True)
faostat_area_harvested = fao_stat_area_harv.merge(crop_class, on='Item')

In [ ]:
CARBON_EMISSION_C3C4_5tahun = xarray.load_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION_C3C4_5tahun_run7.nc", engine="netcdf4")
# CARBON_EMISSION_C3C4_5tahun.isel(time=1).sel(lat=-7.125, lon=109.375).to_pandas()
CARBON_EMISSION_C3C4_5tahun

In [ ]:
gfrac_interp = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-INTERPOLATION-WORLDWIDE-1970_1975.nc", engine="netcdf4")
gfrac_interp

In [ ]:

gfrac = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/GFRAC_GRAPC_Interpolation/GFRAC-RESAMPLE-NEW-1970-2100.nc", engine='netcdf4')
gfrac_ngfbfc = gfrac.coords['NGFBFC'].data.tolist()

static = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="netcdf4")

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['name'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]
    
ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')
ccode_convert = np.zeros((720, 1440), dtype="<U64")

for i in range(720):
    for j in range(1440):
        if ccode_worldwide_int[i][j] in ccode_dict.keys():
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"

In [ ]:

def grid_check_process(grid_luh, grid_gfrac, year, country):

    # kondisi A 
    if len(grid_luh) == 1 and len(grid_gfrac) == 1:
        print("masuk A")
        if (faostat_area_harvested['LUH2Class'] == grid_luh[0]).any():
            # nilai dari fao corps ini belum dipakai
            corps = np.array([(faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                                    (faostat_area_harvested['Year'] == year) &
                                                    (faostat_area_harvested['IMAGE Classification'] == grid_gfrac[0]) &
                                                    (faostat_area_harvested['LUH2Class'] == grid_luh[0])]['Item']), 

                            (faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                                    (faostat_area_harvested['Year'] == year) &
                                                    (faostat_area_harvested['IMAGE Classification'] == grid_gfrac[0]) &
                                                    (faostat_area_harvested['LUH2Class'] == grid_luh[0])]['True Value'])])
            
            emission_intensity = (CARBON_EMISSION_C3C4_5tahun[f'all_to_{grid_luh[0]}_var'].isel(time=1).sel(lat=-7.125, lon=109.375).values \
                                  / (gfrac_interp['gfrac_interp'].isel(time=1).sel(lat=-7.125, lon=109.375).sel(NGFBFC=f'RF {grid_gfrac[0]}').values *\
                                      luh2_static['carea'].sel(lat=-7.125, lon=109.375).values))
            
            return corps, (f"{emission_intensity} ton CO2/ ha")

    # Kondisi B 
    elif len(grid_luh) == 1 and len(grid_gfrac) > 1:
        print("masuk B")
        result = list()
        for idx, ngfbfc in enumerate(grid_gfrac):
            luh2_item = (faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                            (faostat_area_harvested['Year'] == year) &
                                                            (faostat_area_harvested['Area'] == country)]['LUH2Class']).values
            if (grid_luh[0] == luh2_item).any():

                corps = np.array([(faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                            (faostat_area_harvested['Year'] == year) &
                                                            (faostat_area_harvested['Area'] == country) &
                                                            (faostat_area_harvested['LUH2Class'] == grid_luh[0])]['Item']),

                                    (faostat_area_harvested[(faostat_area_harvested['Year'] == year) & 
                                                            (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                            (faostat_area_harvested['Area'] == country) &
                                                            (faostat_area_harvested['LUH2Class'] == grid_luh[0])]['IMAGE Classification']),

                                    (faostat_area_harvested[(faostat_area_harvested['Year'] == year) & 
                                                            (faostat_area_harvested['Area'] == country) &
                                                            (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                            (faostat_area_harvested['LUH2Class'] == grid_luh[0])]['True Value'])])
                
                crops_sum =  faostat_area_harvested[(faostat_area_harvested['Year'] == year) & 
                                                            (faostat_area_harvested['Area'] == country) &
                                                            (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                            (faostat_area_harvested['LUH2Class'] == grid_luh[0])]['True Value'].sum()
                
                crops_carea = (gfrac_interp['gfrac_interp'].isel(time=1).sel(lat=-7.125, lon=109.375).sel(NGFBFC=f'RF {ngfbfc}').values *\
                                    luh2_static['carea'].sel(lat=-7.125, lon=109.375).values)
                if crops_carea != 0:
                    emission_intensity = (CARBON_EMISSION_C3C4_5tahun[f'all_to_{grid_luh[0]}_var'].isel(time=1).sel(lat=-7.125, lon=109.375).values \
                                    / crops_carea)
                else:
                    emission_intensity = 0
                    
                result.append(emission_intensity)

            for x in range(len(corps[0])):
                tabel2 = np.array([corps[0][x], corps[1][x], corps[2][x], (corps[2][x]/crops_sum)*100])
                print(tabel2)
            
            else:
                continue

        for i, emission in enumerate(result):
            print(f"{emission} ton CO2/ ha")

    #Kondisi C
    elif len(grid_luh) > 1 and len(grid_gfrac) == 1:
        print("masuk C")
        sum_proporsi = list()
        tabel2 = list()
        for items in grid_luh:
            corps = np.array([(faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == grid_gfrac[0]) &
                                (faostat_area_harvested['Year'] == year) &
                                (faostat_area_harvested['Area'] == country) &
                                (faostat_area_harvested['LUH2Class'] == items)]['Item']),
                                
                                (faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == grid_gfrac[0]) &
                                (faostat_area_harvested['Year'] == year) &
                                (faostat_area_harvested['Area'] == country) &
                                (faostat_area_harvested['LUH2Class'] == items)]['True Value'])])

            corps_sum = np.array(faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == grid_gfrac[0]) &
                                (faostat_area_harvested['Year'] == year) &
                                (faostat_area_harvested['Area'] == country) &
                                (faostat_area_harvested['LUH2Class'] == items)]['True Value'].sum())
            
            if len(corps[0]) > 1:
                for x in range(0, len(corps)):
                    tabel2.append([corps[0][x], corps[1][x], ((corps[1][x]/corps_sum)*100)])
            else:
                if corps.size == 0:
                    continue
                else:
                    sum_proporsi.append(corps_sum)
                    tabel2.append([corps[0][0], corps[1][0], corps_sum, ((corps[1][0]/corps_sum) * 100)])
        
        crarea_hektar = []
        for idx, i in enumerate(tabel2):
            corps_proportion = np.array([tabel2[idx][0], tabel2[idx][1], tabel2[idx][2], tabel2[idx][3], tabel2[idx][2]/sum(sum_proporsi), tabel2[idx][2]/sum(sum_proporsi) * tabel2[idx][3]])
            print(corps_proportion)
            crops_crarea = float(corps_proportion[5]) * (0.4 * 100) / 100
            crarea_hektar.append(crops_crarea)

        return f"{round(sum(crarea_hektar))} hektar"
        
    #kondisi D1 D2
    elif len(grid_luh) > 1 and len(grid_gfrac) > 1:
        print("masuk D")
        list_mayor_grid = np.array(['Oil, palm fruit', 'Wheat', 'Soybeans', 'Maize', 'Rice'])
        sum_proporsi = list()
        crarea_hektar = list()
        tabel2 = list()
        grid_mayor = list()
        for items in grid_luh:
            for grid in grid_gfrac:
                if grid in list_mayor_grid: #Oil, palm fruit
                    grid_gfrac = gfrac['GFRAC_res'].isel(time=1).sel(lat=-7.125, lon=109.375).sel(NGFBFC="Oil, palm fruit").values
                    grid_corps = grid_gfrac * luh2_static['carea'].sel(lat=-7.125, lon=109.375).values
                    return grid_mayor.append(grid_corps)
                    # continue #diambil nilai major nya 0.4 * 100 carea 
                
                else:
                    corps = np.array([(faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == grid) &
                                    (faostat_area_harvested['Year'] == year) &
                                    (faostat_area_harvested['Area'] == country) &
                                    (faostat_area_harvested['LUH2Class'] == items)]['IMAGE Classification']),

                                    (faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == grid) &
                                    (faostat_area_harvested['Year'] == year) &
                                    (faostat_area_harvested['Area'] == country) &
                                    (faostat_area_harvested['LUH2Class'] == items)]['Item']),
                                    
                                    (faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == grid) &
                                    (faostat_area_harvested['Year'] == year) &
                                    (faostat_area_harvested['Area'] == country) &
                                    (faostat_area_harvested['LUH2Class'] == items)]['True Value'])])
                    
                    corps_sum = np.array(faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == grid) &
                                    (faostat_area_harvested['Year'] == year) &
                                    (faostat_area_harvested['Area'] == country) &
                                    (faostat_area_harvested['LUH2Class'] == items)]['True Value'].sum())
                
                    if len(corps[0]) > 1:
                        sum_proporsi.append(corps_sum)
                        for x in range(0, len(corps[0])):
                            tabel2.append([items[9:], corps[0][x], corps[1][x], corps[2][x], corps_sum, ((corps[2][x]/corps_sum)*100)])
                    else:
                        if corps.size == 0:
                            continue
                        else:
                            sum_proporsi.append(corps_sum)
                            tabel2.append([items[9:], corps[0][0], corps[1][0], corps[2][0], corps_sum, ((corps[2][0]/corps_sum)*100)])

        for idx, grid_corps in enumerate(tabel2):
            corps_proportion = np.array([grid_corps[0], grid_corps[1], grid_corps[2], grid_corps[3], grid_corps[4], grid_corps[5], grid_corps[4]/sum(sum_proporsi), grid_corps[4]/sum(sum_proporsi) * grid_corps[5]])
            print(corps_proportion)
            crops_crarea = float(corps_proportion[5]) * (0.4 * 100) / 100
            crarea_hektar.append(crops_crarea)
        return f"{round(sum(crarea_hektar))} hektar"

### grid jawa tengah

In [ ]:
grid_check_process(grid_luh=["c3per"], grid_gfrac=["Tropical oil crops", "Oil & palm fruit"], year=2017, country='Ghana')

In [ ]:

grid_gfrac = gfrac_interp['GFRAC_interp'].isel(time=1).sel(lat=-7.125, lon=109.375)
# atribut = []

att_ngfbfc = gfrac_interp.coords['NGFBFC'].where(grid_gfrac > 0.0, gfrac_interp['GFRAC_interp'], 0).data
att_ngfbfc

# for idx, value in enumerate(grid_gfrac):
    # print(value.values)
    # if value.any() > 0.0:
        # print(gfrac_interp.coords['NGFBFC'].data.tolist()[idx])
# atribut

In [ ]:
# ates_worldwide.drop(["primf", "primn", "secdf", "secdn", "urban", "pastr", "range", "secmb", "secma"])

crops_list = ["c3ann", "c4ann", "c3per", "c4per", "c3nfx"]
atribut_luh = list()
for idx, value in enumerate(luh2_states_worldwide.isel(time=2).sel(lat=-7.125, lon=109.375).to_pandas().values):
    if value.any() > 0.0:
        attribut = crops_list[idx]
        atribut_luh.append(attribut)
atribut_luh

In [ ]:
for i in range(52):
    if float(luh2_trans_worldwide['secdf_to_c3ann'].isel(time=i).sel(lat=-7.125, lon=109.375).values) > 0.0:
        print(i, luh2_trans_worldwide['secdf_to_c3ann'].isel(time=i).sel(lat=-7.125, lon=109.375).values)


for i in range(52):
    if gfrac['GFRAC_res'].isel(time=i).sel(lat=-7.125, lon=109.375).values.any() > 0.0:
        for idx, value in enumerate(gfrac['GFRAC_res'].isel(time=i).sel(lat=-7.125, lon=109.375).values):
            if value > 0.0:
                ngfbfc_dfrac = [gfrac.coords['NGFBFC'].data.tolist()[idx]]

In [ ]:
    # kondisi A 
    if len(luh2_cat) == 1 and len(ngfbfc_att) == 1:
        jenis_kacang_kacangan += faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                                (faostat_area_harvested['Year'] == year) &
                                                (faostat_area_harvested['IMAGE Classification'] == ngfbfc_att) &
                                                (faostat_area_harvested['LUH2Class'] == luh2_cat)]['Item'].tolist()
        
        crops += faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                                (faostat_area_harvested['Year'] == year) &
                                                (faostat_area_harvested['IMAGE Classification'] == ngfbfc_att) &
                                                (faostat_area_harvested['LUH2Class'] == luh2_cat)]['True Value'].tolist()

    # Kondisi B 
    elif len(luh2_cat) == 1 and len(ngfbfc_att) > 1:

        for ngfbfc in ngfbfc_att:
            jenis_kacang_kacangan += faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                        (faostat_area_harvested['Year'] == year) &
                                        (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                        (faostat_area_harvested['LUH2Class'] == luh2_cat)]['Item'].tolist()
                    
            crops += faostat_area_harvested[(faostat_area_harvested['Year'] == year) & 
                                                        (faostat_area_harvested['Area'] == country) &
                                                        (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                        (faostat_area_harvested['LUH2Class'] == luh2_cat)]['True Value'].tolist()
            
            crops_sum += faostat_area_harvested[(faostat_area_harvested['Year'] == year) & 
                                                 (faostat_area_harvested['Area'] == country) &
                                                 (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                 (faostat_area_harvested['LUH2Class'] == luh2_cat)]['True Value'].sum()
        
    #Kondisi C
    elif len(luh2_cat) > 1 and len(ngfbfc_att) == 1:

        for items in luh2_cat:
            jenis_kacang_kacangan += faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                        (faostat_area_harvested['Year'] == year) &
                                        (faostat_area_harvested['IMAGE Classification'] == ngfbfc_att) &
                                        (faostat_area_harvested['LUH2Class'] == items)]['Item'].tolist()
            
            crops += faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == ngfbfc_att) &
                                (faostat_area_harvested['Year'] == year) &
                                (faostat_area_harvested['Area'] == country) &
                                (faostat_area_harvested['LUH2Class'] == items)]['True Value'].tolist()

            crops_sum += faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == ngfbfc_att) &
                                (faostat_area_harvested['Year'] == year) &
                                (faostat_area_harvested['Area'] == country) &
                                (faostat_area_harvested['LUH2Class'] == items)]['True Value'].sum()
        
    #kondisi D1 D2
    elif len(luh2_cat) > 1 and len(ngfbfc_att) > 1:
        list_mayor_grid = np.array(['Oil, palm fruit', 'Wheat', 'Soybeans', 'Maize', 'Rice'])

        for items in luh2_cat:
            for ngfbfc in ngfbfc_att:

                if ngfbfc in list_mayor_grid:
                    jenis_kacang_kacangan += faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                        (faostat_area_harvested['Year'] == year) &
                                        (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                        (faostat_area_harvested['LUH2Class'] == items)]['Item'].tolist()
                    
                    crops += [gfrac_interp['GFRAC_interp'].isel(time=time, lat=lat, lon=lon).sel(NGFBFC=ngfbfc).values * luh2_static['carea'].isel(lat=lat, lon=lon).values]

                if ngfbfc is not list_mayor_grid:
                    jenis_kacang_kacangan += faostat_area_harvested[(faostat_area_harvested['Area'] == country) &
                                        (faostat_area_harvested['Year'] == year) &
                                        (faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                        (faostat_area_harvested['LUH2Class'] == items)]['Item'].tolist()
                    
                    crops += faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                                    (faostat_area_harvested['Year'] == year) &
                                                    (faostat_area_harvested['Area'] == country) &
                                                    (faostat_area_harvested['LUH2Class'] == items)]['True Value'].tolist()
                
                    crops_sum += faostat_area_harvested[(faostat_area_harvested['IMAGE Classification'] == ngfbfc) &
                                    (faostat_area_harvested['Year'] == year) &
                                    (faostat_area_harvested['Area'] == country) &
                                    (faostat_area_harvested['LUH2Class'] == items)]['True Value'].sum()